# 上传mask文件夹

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving test.zip to test.zip


In [ ]:
import zipfile

zip_file_name = "test1.zip" # image原图像 mask真实值 test合成图像

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall()

# 保存mask

In [ ]:
from PIL import Image
import os

# 指定mask文件夹路径
mask_folder_path = "mask_new/"  # 根据你实际上传的路径进行修改
i = 0

# 遍历mask文件夹及其子文件夹中所有文件
for root, dirs, files in os.walk(mask_folder_path):
    for file in files:
        # 判断文件是否为图片文件（可以根据实际需要添加其他图片格式的判断）
        if file.endswith((".png", ".jpg", ".jpeg", ".bmp")):
            image_path = os.path.join(root, file)
            img = Image.open(image_path)

            # 将图片转换为灰度图像
            img_gray = img.convert("L")

            # 将灰度图像转换为二值化图像
            img_binary = img_gray.point(lambda x: 0 if x < 64 else 255, '1')

            # 将二值化图像转换为RGB图像
            img_rgb = img_binary.convert("RGB")

            # 保存处理后的图片，覆盖原来的图片
            img_rgb.save(image_path)
            i = i + 1
print(i)
print("图片处理完成！")

63
图片处理完成！


保存的mask为二值图像

# 下载mask

In [ ]:
import shutil

# 指定要下载的文件夹路径
mask_folder_path = "mask01"  # 根据你实际的文件夹路径进行修改

# 指定zip文件名
zip_file_name = "mask01.zip"

# 将文件夹打包成zip文件
shutil.make_archive("mask01", 'zip', mask_folder_path)

from google.colab import files

# 下载zip文件
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pip install opencv-python

# 创建合成数据集

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# 获取mask文件夹中的图片文件列表
mask_image_files = os.listdir('mask/')

# 创建颜色选项列表
color_options = [
    (50, 50, 50),  # 深黑色
    (200, 200, 200),  # 浅灰色
    (135, 136, 122),  # 棕色
    (100, 100, 100),  # 浅黑色
    (70, 80, 100),  # 深蓝色
    (134, 127, 117)  # 深红色

]

# 遍历每个mask文件
for mask_file in mask_image_files:
    mask_image_path = os.path.join('mask', mask_file)
    image_file = mask_file  # 假设mask和image文件名一致
    if image_file in os.listdir('image'):
        image_A_path = os.path.join('image', image_file)
        image_B_path = mask_image_path

        # 读取图像A（image）和图像B（mask）
        image_A = cv2.imread(image_A_path, cv2.IMREAD_COLOR)
        image_B = cv2.imread(image_B_path, cv2.IMREAD_COLOR)

        # 将图像B转换为灰度图像
        gray_image = cv2.cvtColor(image_B, cv2.COLOR_BGR2GRAY)

        # 使用阈值化操作获取白色区域的二值掩码
        _, thresholded = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)

        # 寻找白色区域的连通域
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 输出连通域的个数
        print(f"图像 {mask_file} 中的连通域个数:", len(contours))

        # 遍历test文件夹中的每张图片
        test_image_files = os.listdir('test1')
        for test_image_file in test_image_files:
            # 读取test文件夹中的图片作为image_C
            image_C = cv2.imread(os.path.join('test1', test_image_file), cv2.IMREAD_COLOR)

            # 创建空白图像E
            image_E = np.zeros_like(image_C)

            # 遍历每个连通域
            for contour in contours:
                # 创建当前连通域的空白掩码
                mask = np.zeros_like(gray_image)

                # 在当前连通域的掩码上绘制连通域的白色区域
                cv2.drawContours(mask, [contour], -1, (255), thickness=cv2.FILLED)

                # 获取当前连通域的白色像素索引
                white_pixels = np.where(mask == 255)

                # 随机变换参数
                scale_factor = np.random.uniform(0.8, 1.2)  # 随机缩放因子
                rotation_angle = np.random.uniform(-60, 60)  # 随机旋转角度
                translation_x = np.random.randint(15, 65)  # 随机水平平移
                translation_y = np.random.randint(15, 65)  # 随机垂直平移
                selected_color = color_options[np.random.randint(len(color_options))]

                # 计算当前连通域的矩
                M = cv2.moments(contour)
                contour_center_x = int(M["m10"] / M["m00"])
                contour_center_y = int(M["m01"] / M["m00"])

                # 创建仿射变换矩阵
                M = cv2.getRotationMatrix2D((contour_center_x, contour_center_y), rotation_angle, scale_factor)
                M[:, 2] += (translation_x, translation_y)

                # 对当前连通域进行仿射变换和颜色变换
                transformed_contour = cv2.transform(contour, M)
                transformed_contour = transformed_contour.astype(np.int32)
                transformed_contour += (translation_x, translation_y)

                # 绘制变换后的连通域
                cv2.drawContours(image_C, [transformed_contour], -1, selected_color, thickness=cv2.FILLED)

                # 在image_E中将连通域的区域置为白色
                cv2.drawContours(image_E, [transformed_contour], -1, (255, 255, 255), thickness=cv2.FILLED)

            # 从 mask_file 中提取文件名部分（去掉后缀）
            mask_file_name = os.path.splitext(mask_file)[0]

            # 构建保存图片的文件路径，包括 mask 的名称和 test_image_file
            output_image_A_path = os.path.join('image01', f'{mask_file_name}_{test_image_file}')
            output_image_E_path = os.path.join('mask01', f'{mask_file_name}_{test_image_file}')

            cv2.imwrite(output_image_A_path, image_C)
            cv2.imwrite(output_image_E_path, image_E)

print("处理完成并保存图片。")

图像 P110.jpg 中的连通域个数: 7
图像 P091.jpg 中的连通域个数: 10
图像 P146.jpg 中的连通域个数: 42
图像 P077.jpg 中的连通域个数: 5
图像 P008.jpg 中的连通域个数: 4
图像 P062.jpg 中的连通域个数: 21
图像 P148.jpg 中的连通域个数: 29
图像 P123.jpg 中的连通域个数: 17
图像 P162.jpg 中的连通域个数: 24
图像 P118.jpg 中的连通域个数: 10
图像 P095.jpg 中的连通域个数: 5
图像 P104.jpg 中的连通域个数: 16
图像 P084.jpg 中的连通域个数: 4
图像 P161.jpg 中的连通域个数: 24
图像 P155.jpg 中的连通域个数: 32
图像 P158.jpg 中的连通域个数: 17
图像 P153.jpg 中的连通域个数: 19
图像 Control.jpg 中的连通域个数: 4
图像 P138.jpg 中的连通域个数: 5
图像 P105.jpg 中的连通域个数: 13
图像 P152.jpg 中的连通域个数: 24
图像 P128.jpg 中的连通域个数: 3
图像 P075.jpg 中的连通域个数: 4
图像 P103.jpg 中的连通域个数: 22
图像 P053.jpg 中的连通域个数: 14
图像 P121.jpg 中的连通域个数: 27
图像 P083.jpg 中的连通域个数: 5
图像 P154.jpg 中的连通域个数: 27
图像 P140.jpg 中的连通域个数: 11
图像 No ID 1.jpg 中的连通域个数: 14
图像 P137.jpg 中的连通域个数: 23
图像 P090.jpg 中的连通域个数: 36
图像 P107.jpg 中的连通域个数: 5
图像 P080.jpg 中的连通域个数: 5
图像 P067.jpg 中的连通域个数: 5
图像 P127.jpg 中的连通域个数: 31
图像 No ID 2.jpg 中的连通域个数: 43
图像 No ID 3.jpg 中的连通域个数: 20
图像 P088.jpg 中的连通域个数: 39
图像 P101.jpg 中的连通域个数: 10
图像 P115.jpg 中的连通域个数: 15
图像 P038.jpg 中的连通域

共创建630张

有三张图像生成的合成数据集效果不好，故删除，图像数据为p088,p090,p148

所以使用的合成数据集一共600张图像